In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [27]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [28]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [29]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [30]:
def weighted_random_recommendation(items_weights, n=5):
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n,replace=False, p = weights)
    
    
    return recs.tolist()

In [31]:
#По кол-ву проданных штук
items_weights_quantity = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights_quantity['weight'] = items_weights_quantity['quantity'].apply(lambda x: np.log(x) if x != 0 else x)
items_weights_quantity['weight'] = items_weights_quantity['weight'] / items_weights_quantity['weight'].sum()

#По сумме продаж
items_weights_sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights_sales['weight'] = items_weights_sales['sales_value'].apply(lambda x: abs(np.log(x)) if x != 0 else x)
items_weights_sales['weight'] = items_weights_sales['weight'] / items_weights_sales['weight'].sum()

#По кол-ву отдельных продаж
items_weights_counts = data_train.groupby('item_id')['sales_value'].count().reset_index()
items_weights_counts['weight'] = items_weights_counts['sales_value'].apply(lambda x: abs(np.log(x + 1)) if x != 0 else x)
items_weights_counts['weight'] = items_weights_counts['weight'] / items_weights_counts['weight'].sum()
items_weights_counts

,item_id,sales_value,weight
0,25671,3,0.000009
1,26081,1,0.000004
2,26093,1,0.000004
3,26190,1,0.000004
4,26355,1,0.000004
...,...,...,...
86860,17381856,1,0.000004
86861,17382205,1,0.000004
86862,17383227,1,0.000004
86863,17827644,1,0.000004


In [32]:
%%time

result['weighted_random_recommendation_quantity'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_quantity, n=5))
result['weighted_random_recommendation_sales'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_sales, n=5))
result['weighted_random_recommendation_counts'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_counts, n=5))

Wall time: 5.6 s


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [33]:
result_webinar = pd.read_csv('preds.csv')
result_webinar.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[823788, 1061518, 1209266, 1555146, 831602]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15573262, 15927757, 938920, 883433, 963378]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[15926977, 899682, 2007534, 1051001, 6979786]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[79283, 932412, 4001980, 12987733, 1301566]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1107402, 1079228, 839346, 859700, 917031]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [34]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [35]:
# Метрика по семинару
precision_all = []
result_webinar['actual'] = result_webinar['actual'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
for column in result_webinar.columns[2:]:
  result_webinar[column] = result_webinar[column].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
  precision_all.append(round(result_webinar.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 4))

In [36]:
for column in result.columns[2:]:
  precision_all.append(round(result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 4))

In [37]:
columns = result_webinar.columns[2:].to_list() + result.columns[2:].to_list()
pd.Series(data = precision_all, index = columns)

random_recommendation                      0.0005
popular_recommendation                     0.1552
itemitem                                   0.1369
cosine                                     0.1329
tfidf                                      0.1390
own_purchases                              0.1797
weighted_random_recommendation_quantity    0.0021
weighted_random_recommendation_sales       0.0018
weighted_random_recommendation_counts      0.0018
dtype: float64

Лучшее качество показывает алгоритм ItemItemRecommender с к=1, т.к. он рекоммендует пользователю самые популярные его покупки.

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [38]:
def random_recommendation(items, n=5):

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [39]:
%%time
top5000 = data_train.groupby('item_id')['quantity'].sum().reset_index()
top5000.sort_values('quantity', ascending=False, inplace = True)
items = top5000.head(5000).item_id.to_list()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 868 ms


,user_id,actual,weighted_random_recommendation_quantity,weighted_random_recommendation_sales,weighted_random_recommendation_counts,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9858863, 12384292, 1011215, 827287, 1023693]","[9266843, 9655212, 1007195, 6463724, 1071077]","[45478, 1038163, 905690, 852359, 1859699]","[9934800, 857055, 1093730, 9879409, 909750]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[940207, 1050272, 10204277, 938562, 961785]","[13039855, 990688, 5996083, 1648373, 6534178]","[7130264, 1127258, 1047262, 1134803, 12582096]","[982833, 983753, 879734, 837098, 1068292]"


In [40]:
precision_all.append(round(result.apply(lambda x: precision_at_k(x['random_recommendation_5000'], x['actual'],  5), axis=1).mean(), 4))

In [41]:
columns = result_webinar.columns[2:].to_list() + result.columns[2:].to_list()
all_metrics = pd.Series(data = precision_all, index = columns)

In [42]:
all_metrics[['random_recommendation', 'random_recommendation_5000']]

random_recommendation         0.0005
random_recommendation_5000    0.0065
dtype: float64

Baseline используя Top-5000 для рандом рекомендации улучшился

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? - это условное обозначение товаров, которые не входят в топ 5000 товаров.
Зачем он нужен? - если мы удалим все товары, которые не входят в топ 5000, то остануться пользователи, которые ничего не покупали 
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону? - мы ухудшаем рекомендацию, так как этот товар будет рекомендоваться с большой частотой   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

In [43]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [44]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6534178

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.328651460584234

In [46]:
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556],
      dtype=int64)

In [47]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [57]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 812 ms


In [58]:
%%time

result['itemitem_top1_k1'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 64.8 ms


In [59]:
result.head(3)

,user_id,actual,weighted_random_recommendation_quantity,weighted_random_recommendation_sales,weighted_random_recommendation_counts,random_recommendation_5000,itemitem_top1,itemitem_top1_k1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9858863, 12384292, 1011215, 827287, 1023693]","[9266843, 9655212, 1007195, 6463724, 1071077]","[45478, 1038163, 905690, 852359, 1859699]","[9934800, 857055, 1093730, 9879409, 909750]","[6534178, 1082185, 981760, 1127831, 995242]","[6534178, 1029743, 995785, 866227, 12810393]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[940207, 1050272, 10204277, 938562, 961785]","[13039855, 990688, 5996083, 1648373, 6534178]","[7130264, 1127258, 1047262, 1134803, 12582096]","[982833, 983753, 879734, 837098, 1068292]","[6534178, 1082185, 981760, 1098066, 826249]","[6534178, 9526410, 12810393, 9527290, 8090521]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[827050, 6463717, 1036249, 8091526, 12811581]","[1014247, 1109357, 15717279, 15572718, 853091]","[977279, 1067787, 10312380, 59668, 1118963]","[844441, 836281, 910473, 1029479, 860167]","[6534178, 1082185, 981760, 1127831, 995242]","[6534178, 1029743, 995785, 866227, 9526410]"


In [60]:
precision_all.append(round(result.apply(lambda x: precision_at_k(x['itemitem_top1_k1'], x['actual'],  5), axis=1).mean(), 4))

In [61]:
columns = result_webinar.columns[2:].to_list() + result.columns[2:].to_list()
all_metrics = pd.Series(data = precision_all, index = columns)
all_metrics = pd.Series(data = precision_all, index = columns)


In [62]:
all_metrics

random_recommendation                      0.0005
popular_recommendation                     0.1552
itemitem                                   0.1369
cosine                                     0.1329
tfidf                                      0.1390
own_purchases                              0.1797
weighted_random_recommendation_quantity    0.0021
weighted_random_recommendation_sales       0.0018
weighted_random_recommendation_counts      0.0018
random_recommendation_5000                 0.0065
itemitem_top1                              0.1749
itemitem_top1_k1                           0.1246
dtype: float64

Попробовала заменить товары не из топ 5000 на самый популярный товар. В результате, алгоритм itemitem_top1 с к=5 показал результат хуже, чем с 999999 и к=1, но лучше чем с 999999 и к=5. При попытке улучшить результат itemitem_top1 с применением к=1, результат алгоритма itemitem_top1_k1 сильно ухудшился. 

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.